In [19]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
import os

file_name = "ballscrew_s.xlsx"
absolute_path = os.path.abspath(file_name)

print(absolute_path)

C:\Users\Sanam\git\Thesis_ErrorDetection\ballscrew_s.xlsx


In [21]:
# Load your data from the Excel files
data = pd.read_excel('ballscrew_s.xlsx', header=None)
#data = pd.read_excel('leadscrew_s.xlsx', header=None)

In [23]:
data


,0,1,2,3
0,commanded position,theoretical position,actual position,error
1,0.09794,0.1,0.1,0.00206
2,-1.4897,-1.5,-1.45,-0.0397
3,5.99082,6,5.95,-0.04082
4,-8.5085,-8.5,-8.45001,-0.05849
...,...,...,...,...
14996,-8.58654,-8.575,-8.525,-0.06154
14997,-7.5981,-7.6,-7.65,0.0519
14998,7.49363,7.5,7.5,0.00637
14999,9.39613,9.4,9.4,0.00387


In [28]:
# Extract the features (X) and target (Y)
X = data.iloc[:, :-1].values  # Use all columns except the last one as features
Y = data.iloc[:, -1].values  # Use the last column as the target

In [25]:
# Normalize the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X)
Y = scaler_Y.fit_transform(Y.reshape(-1, 1))

ValueError: could not convert string to float: 'commanded position'

In [ ]:
# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# Define the neural network
model = Sequential()
model.add(Dense(8, input_dim=X.shape[1], activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='tanh'))  # Assuming it's a regression task with one output

In [ ]:
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1)

In [ ]:
# Evaluate the model
MSE_train = np.mean(np.square(model.predict(X_train) - Y_train))
MSE_test = np.mean(np.square(model.predict(X_test) - Y_test))
print('MSE on training data:', MSE_train)
print('MSE on testing data:', MSE_test)

In [22]:
# Extract the features (X) and target (Y)
X = data.iloc[:, :-1].values  # Use all columns except the last one as features
Y = data.iloc[:, -1].values  # Use the last column as the target

# Normalize the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X)
Y = scaler_Y.fit_transform(Y.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the neural network
model = Sequential()
model.add(Dense(8, input_dim=X.shape[1], activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='tanh'))  # Assuming it's a regression task with one output

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate the model
MSE_train = np.mean(np.square(model.predict(X_train) - Y_train))
MSE_test = np.mean(np.square(model.predict(X_test) - Y_test))
print('MSE on training data:', MSE_train)
print('MSE on testing data:', MSE_test)


ValueError: could not convert string to float: 'commanded position'

In [ ]:
# Extract the features (X) and target (Y)
X = data.iloc[:, :-1].values  # Use all columns except the last one as features
Y = data.iloc[:, -1].values  # Use the last column as the target

# Normalize the data
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X)
Y = scaler_Y.fit_transform(Y.reshape(-1, 1))

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the neural network
model = Sequential()
model.add(Dense(8, input_dim=X.shape[1], activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='tanh'))  # Assuming it's a regression task with one output

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate the model
MSE_train = np.mean(np.square(model.predict(X_train) - Y_train))
MSE_test = np.mean(np.square(model.predict(X_test) - Y_test))
print('MSE on training data:', MSE_train)
print('MSE on testing data:', MSE_test)
